# **TP ModIA - Analyse de sensibilité et indices de Sobol's**

## **0) Framework**

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import random as rd
import scipy.stats as sstats

La sortie $Y$ sera vu comme une fonction de $D \in \mathbb{N}$ variables (potentiellement) explicatives, notées $(X_1, \dots, X_D)$. Pour modéliser la boite noire, on utilisera la Sobol's G-function. Pour un $d \le D$ et un $(a_i)_{1\le i\le d}$ (dont toutes les coordonnées sont différentes de -1), on définit cette fonction par : 
$$G : x \mapsto \prod_{i=1}^d \frac{|4x^{(i)}i -2|+a_i}{1+a_i}$$

In [3]:
def ComputeGFunction( a ):
    """
    :a:      Array of size (dim_d)
    :return: Function f, 
             For an input x, array of size (dim_D, n_sample), 
             f(x) returns y, array of size (n_sample), answer of x through G-function.
                          
    """
    dim_d = a.shape[0]
    return lambda X : np.prod(  ( np.abs( 4 * X[ :dim_d, : ] - 2 ) + a.reshape(-1, 1) ) 
                                / ( 1 + a.reshape(-1, 1) ), 
                              axis = 0 )

L'avantage de ce modèle réside dans notre capacité à évaluer les indices de Sobol', particulièrement dans le cas où la variable $X$ suit une loi uniforme sur $[0,1]$. Dans la suite, on travaillera qu'avec les indices de premier ordre :

$$ S_{\mathrm{Sobol'}}^{(i)} = \frac{\frac{1}{3\left(a_i +1\right)^2}}{\prod_{j=1}^d \left(1 + \frac{1}{3\left(a_j +1\right)^2}\right) - 1 }$$

On travaillera principalement avec les vecteurs $a$ suivant :

In [4]:
dim_d = 8

a1 = np.ones( dim_d ) * 0
a2 = np.ones( dim_d ) * 99
a3 = np.array( [ 0, 1, 4, 4, 7, 9, 99, 99 ] )

   

---


   ##    **I) Estimation des indices de Sobol's**

  ###    **1) La méthode d'estimation'** 


L'estimateur des indices de Sobol' pour une entrée $X^{(k)} = X$, est le suivant :
$$\widehat{S}_{\mathrm{Sobol'}}^{(i)} = \frac{\frac{1}{n}\sum_{j=1}^nY_jY_{N(j)} - \left(\frac{1}{n}\sum_{j=1}^n Y_j\right)^2}{\frac{1}{n}\sum_{j=1}^nY_j^2 - \left(\frac{1}{n}\sum_{j=1}^n Y_j\right)^2}$$
où $N(j)$ est l'indice $1 \le i \le n$ tel que l'échantillon $X_i$ soit le plus proche voisin de l'échantillon $X_j$ (à droite).

Pour coder cette permutation, on utilisera le code suivant :

In [5]:
def ComputeNNPermutation( X ):
  
  permutation         = np.argsort( X, axis = 1 )
  permutation_shifted = np.roll( permutation, shift = -1, axis = 1)
  rank                = np.argsort( permutation, axis = 1 )

  N = np.take_along_axis( permutation_shifted, rank, axis = 1)

  return N



1.   Coder une fonction qui calcule l'estimateur proposé ci-dessus.

In [6]:
def EstimateSobolIndices( inputs, output ):
  n_sample = inputs.shape[1]
  dim_D = inputs.shape[0]
  outputs = np.ones((dim_D,1)) @ output.reshape((1,n_sample))

  NN_permutations = ComputeNNPermutation(inputs)
  sorted_outputs = np.take_along_axis(outputs, NN_permutations, axis = 1)

  term1 = outputs*sorted_outputs
  term2 = outputs
  term3 = outputs**2

  numerator = np.mean(term1, axis = 1) - np.mean(term2, axis = 1)**2
  denominator = np.mean(term3, axis = 1) - np.mean(term2, axis = 1)**2
  est = numerator/denominator


  return est


2.   Construire un algorithme prenant un échantillonnage des entrées, un échantillonnage de la sortie et un paramètre servant à controler le critère de sélection (évacuant les entrées d'indice sous un certain pourcentage par exemple), et qui ne renvoit seulement les entrées jugées pertinentes (classées par ordre d'importance dans l'idéal), avec les indices estimés.

In [ ]:
def MakeSensitivityAnalysis( inputs, output, tolerance = 0.01):

  sensitivity_indices = EstimateSobolIndices(inputs, output)

  sorted_features = np.argsort(sensitivity_indices)
  sorted_features = np.flip(sorted_features)
  sorted_indices = sensitivity_indices[sorted_features]


  relevant_argument = np.argwhere(sorted_indices > tolerance)
  relevant_indices = sorted_indices[relevant_argument]
  relevant_features = sorted_features[relevant_argument]
  

  return relevant_indices, relevant_features


###  **2 ) Aspects pratiques**



1.   Appliquer l'algorithme de la première partie au résultat de la G-function pour les trois valeurs de $a$, proposées plus haut, pour de grandes valeurs de $N$.


> a. Conceptuellement, comment différencier la sensibilité des entrées entre les cas 1 et 2 ?

> b. Dans le cas 3, le résultat de sensibilité etait-il prévisible ? Pourquoi certaine variable ne sont pas détecté dans ce cas là ?





In [ ]:
## Case 1

In [ ]:
## Case 2

In [ ]:
## Case 3

2.   A partir de second cas $a_2$, afficher l'indice de sensibilité de $X_1$ et de $X_9$ en fonction de la taille de l'échantillon. 



In [ ]:
## Evolution de l'estimateur pour X^{(1)}

In [ ]:
## Evolution de l'estimateur pour X^{(9)}

## **II) Estimation des indices de Cramèr-von-Mises** 

  ###    **1) La méthode d'estimation'** 


Initialement, cette méthodes d'estimation des indices de sensibilité par les rangs a été popularisée par Sourav Chatterjee pour les indices de Cramèr-von-Mises :



Si on réarrange les paires $\left(X_i,Y_i\right)_i$ de façon à ce que les $\left(X_{(i)},Y_{(i)}\right)$ vérifient :
$$X_{(1)} \le X_{(2)} \le \dots \le X_{(n)} \ ,$$
alors on note $r_j$ le rang de $Y_{(j)}$, i.e. le nombre d'échantillon $Y_{(i)}$ inférieur à $Y_{(j)}$. 

L'estimateur vaut alors :
$$\widehat{S}_{\mathrm{CvM}}^{(i)} = 1 - \frac{3 \sum_{j=1}^n | r_{j+1} - r_j | }{n^2 - 1} \ .$$

On peut montrer qu'il est asymptotiquement égal à 
$$\widetilde{S}_{\mathrm{CvM}}^{(i)} = \frac{\frac{1}{n} \sum_{j=1}^n \min\left(F_n(Y_j),F_n(Y_{N(j)}) \right) - \frac{1}{n}\sum_{j=1}^n F_n(Y_j)^2}{\frac{1}{n}\sum_{j=1}^n F_n(Y_j) \left(1- F_n(Y_j) \right)}$$
où $N(j)$ est l'indice $1 \le i \le n$ tel que $X_i$ soit le plus proche voisin de $X_j$ (à droite) et où $F_n$ désigne la fonction de répartition empirique de $Y$. 

On retrouve d'ailleurs la formulation proche des estimateurs des indices de Sobol'.

1.   Coder une fonction qui calcule un des estimateurs proposés ci-dessus. 

In [ ]:
def EstimateCvMIndices(inputs, output) :

    
    return estimator

A noter qu'on possède un Théorème Central Limite sous l'hypothèse $S_{\mathrm{CvM}}^{(i)} = 0$ :
$$\sqrt{n}\widehat{S}_{\mathrm{CvM}}^{(i)} \longrightarrow \mathcal{N}(0,2/5)$$


2.   Etant donné le TCL sur cet estimateur sous l'hypothèse nulle "S_{\mathrm{CvM}}^{(i)} = 0", proposer un critère de rejet des variables peu sensibles.

3.   Utiliser ce critère pour construire un algorithme prenant un échantillonnage des entrées, un échantillonnage de la sortie et un paramètre servant à controler le critère de sélection (confiance du test par exemple), et qui ne renvoit seulement les entrées jugées pertinentes (classées par ordre d'importance dans l'idéal), avec les indices associés.

In [ ]:
def MakeSensitivityAnalysis( inputs, output, tolerance = 0.01, indices = 'sobol', confidence_cvm = None):
  
  

  return relevant_indices, relevant_features

### **2) Aspects pratiques**



1.   Appliquer l'algorithme de la première partie au résultat de la G-function pour les trois valeurs de $a$, proposées plus haut, pour de grandes valeurs de $N$. Les cas 1 et 2 ne paraissent-ils pas surprenant ?



In [ ]:
## Case 1
               

In [ ]:
## Case 2

In [ ]:
## Case 3

2.   A partir de second cas (a2), afficher l'indice de sensibilité de $X_1$ et de $X_9$ en fonction de la taille de l'échantillon. Pour $X_9$, afficher en plus la courbe de l'intervalle de confiance. Que dire de la vitesse de convergence?



In [ ]:
## Evolution de l'estimateur pour X^{(1)}

In [ ]:
## Evolution de l'estimateur pour X^{(9)}